In [ ]:

import pandas as pd
import numpy as np
# 1. LOAD DATASETS
# Replace these strings with your actual file paths
league = pd.read_csv('league.csv')
new_batting = pd.read_csv('new_batting.csv')
new_bowling = pd.read_csv('new_bowling.csv')
new_fielding = pd.read_csv('new_fielding.csv')
player_details = pd.read_csv('Player Details.csv')

In [ ]:


# Note: The sub-module functions (calculate_professional_pps, calculate_consistency_with_form, etc.)
# are assumed to be defined as provided in the previous context. If needed, they can be included here.
# For correctness, we've ensured all column references match the data model exactly, with stripping for whitespace.

def calculate_professional_pps(df_bat, df_bowl, df_league, df_player):
    # --- 1. PRE-PROCESSING & LEVEL MAPPING ---
    # Resistance Factors: Elite(1.5), Capped(1.0), Uncapped(0.5)
    level_map = {'Uncapped': 0.5, 'Capped': 1.0, 'Elite': 1.5}
    df_player = df_player.copy()
    df_player['level_res'] = df_player['Level'].map(level_map).fillna(1.0)
   
    league_mini = df_league[['match_id', 'home team', 'visiting team', 'home_importance', 'visitor_importance']].copy()
    # --- 2. BATTING PPS CALCULATION ---
    bat = df_bat.merge(league_mini, on='match_id', how='left')
    bat = bat.merge(df_player[['Player_id', 'level_res']], left_on='playerid', right_on='Player_id', how='left')
    bat['team_imp'] = np.where(bat['Team_Name'] == bat['home team'], bat['home_importance'], bat['visitor_importance'])
    bat['eps'] = (bat['Fall of Wicket'].fillna(0) + 1) / (bat['Over of wicket'].fillna(0) + 1)
    bat['sm'] = np.exp(bat['team_imp'] / 5)
    bat['output'] = bat['Runs'] * (bat['strike Rate'] / 100)
    bat['m_pps'] = (bat['output'] * bat['eps'] * bat['sm']) / bat['level_res']
    # --- 3. BOWLING PPS CALCULATION ---
    bowl = df_bowl.merge(league_mini, left_on='matchId', right_on='match_id', how='left')
    bowl = bowl.merge(df_player[['Player_id', 'level_res']], left_on='Bowler_id', right_on='Player_id', how='left')
    bowl['team_imp'] = np.where(bowl['Team'] == bowl['home team'], bowl['home_importance'], bowl['visitor_importance'])
    bowl['eps'] = bowl['Innings'].map({1: 1.0, 2: 1.4})
    bowl['sm'] = np.exp(bowl['team_imp'] / 5)
    bowl['output'] = (bowl['Wickets'] * 25) + bowl['Dots']
    bowl['m_pps'] = (bowl['output'] * bowl['eps'] * bowl['sm']) / bowl['level_res']
    # --- 4. DATA CONSOLIDATION ---
    bat_final = pd.DataFrame({'playerid': bat['playerid'], 'match_id': bat['match_id'], 'm_pps': bat['m_pps']})
    bowl_final = pd.DataFrame({'playerid': bowl['Bowler_id'], 'match_id': bowl['match_id'], 'm_pps': bowl['m_pps']})
    all_match_pps = pd.concat([bat_final, bowl_final], axis=0, ignore_index=True)
   
    # --- 5. HISTORICAL REPUTATION (EMA) ---
    all_match_pps = all_match_pps.sort_values(['playerid', 'match_id'])
    all_match_pps['pps_ema'] = all_match_pps.groupby('playerid')['m_pps'].transform(lambda x: x.ewm(span=5).mean())
    # --- 6. MERGE TO MASTER (247+ PLAYERS) ---
    active_reps = all_match_pps.groupby('playerid')['pps_ema'].last().reset_index()
    final_roster = df_player.merge(active_reps, left_on='Player_id', right_on='playerid', how='left')
    final_roster['pps_ema'] = final_roster['pps_ema'].fillna(0)
    # --- 7. SCALING WITH SELECTION BASELINE (30) ---
    # We recognize that just being on the roster implies a standard level of ability
    SELECTION_BASE = 30
   
    active_mask = final_roster['pps_ema'] > 0
    mu = final_roster.loc[active_mask, 'pps_ema'].mean()
    sigma = final_roster.loc[active_mask, 'pps_ema'].std()
   
    # Calculate Z-score for performance differentiation
    final_roster['z'] = (final_roster['pps_ema'] - mu) / (sigma + 1e-9)
   
    # Apply Sigmoid on the remaining 70% of the scale
    final_roster['PPS_Score'] = SELECTION_BASE + ( (100 - SELECTION_BASE) / (1 + np.exp(-final_roster['z'])) )
   
    # Players who are selected but have no match data yet stay at the SELECTION_BASE
    final_roster.loc[~active_mask, 'PPS_Score'] = SELECTION_BASE
    return final_roster[['Player_id', 'Player_name', 'Team', 'Level', 'PPS_Score']].sort_values(by='PPS_Score', ascending=False)

def calculate_consistency_with_form(df_bat, df_bowl, df_player):
    # --- 1. PRE-PROCESSING & LEVEL MAPPING ---
    level_map = {'Uncapped': 0.5, 'Capped': 1.0, 'Elite': 1.5}
    df_player = df_player.copy()
    df_player['level_res'] = df_player['Level'].map(level_map).fillna(1.0)
   
    # --- 2. PREPARE MATCH OUTPUTS ---
    bat_sub = pd.DataFrame({'pid': df_bat['playerid'], 'mid': df_bat['match_id'],
                          'out': df_bat['Runs'] * (df_bat['strike Rate'] / 100)})
    bowl_sub = pd.DataFrame({'pid': df_bowl['Bowler_id'], 'mid': df_bowl['matchId'],
                           'out': (df_bowl['Wickets'] * 25) + df_bowl['Dots']})
   
    all_perf = pd.concat([bat_sub, bowl_sub], axis=0, ignore_index=True).sort_values(['pid', 'mid'])
    # --- 3. CALCULATE FORM (EMA) & RECENT VOLATILITY ---
    # EMA represents 'Form'
    all_perf['form_ema'] = all_perf.groupby('pid')['out'].transform(lambda x: x.ewm(span=5).mean())
    # Rolling STD of last 5 matches represents 'Recent Stability'
    all_perf['rolling_std'] = all_perf.groupby('pid')['out'].transform(lambda x: x.rolling(window=5, min_periods=1).std())
    # --- 4. CALCULATE STABILITY INDEX ---
    final_stats = all_perf.groupby('pid').last().reset_index()
    final_stats = final_stats.merge(df_player[['Player_id', 'level_res']], left_on='pid', right_on='Player_id', how='right')
   
    # Stability Index = Form / ((Volatility + 1) * Level Resistance)
    final_stats['volatility'] = (final_stats['rolling_std'].fillna(final_stats['form_ema'] * 0.5)) / (final_stats['form_ema'] + 1)
    final_stats['esi'] = final_stats['form_ema'] / ((final_stats['volatility'] + 1) * final_stats['level_res'])
    final_stats['esi'] = final_stats['esi'].fillna(0)
    # --- 5. SIGMOID SCALING WITH BASELINE 30 ---
    active = final_stats[final_stats['esi'] > 0]
    mu, sigma = active['esi'].mean(), active['esi'].std()
   
    final_stats['z'] = (final_stats['esi'] - mu) / (sigma + 1e-9)
    final_stats['Consistency_Score'] = 30 + (70 / (1 + np.exp(-final_stats['z'])))
   
    # Non-participants or 0-impact players stay at baseline 30
    final_stats.loc[final_stats['esi'] <= 0, 'Consistency_Score'] = 30
    return final_stats[['Player_id', 'Consistency_Score']].merge(df_player, on='Player_id')

def calculate_phased_final(df_bat, df_bowl, df_player):
    # --- 1. SETUP & LEVEL RESISTANCE ---
    # Ensure column names are standard
    df_player.columns = [c.strip() for c in df_player.columns]
   
    level_map = {'Uncapped': 0.5, 'Capped': 1.0, 'Elite': 1.5}
    master = df_player.copy()
    master['level_res'] = master['Level'].map(level_map).fillna(1.0)
   
    # --- 2. DEFINE PHASE OUTPUTS ---
    # Batting (Match-level proxies for phases)
    df_bat['p_open'] = (df_bat['strike Rate'] * 0.4 + df_bat['fours'] * 5)
    df_bat['p_mid'] = (df_bat['Runs'] * 1.2)
    df_bat['p_death'] = (df_bat['sixes'] * 15 + df_bat['strike Rate'] * 0.6)
   
    # Bowling (Match-level proxies for phases)
    df_bowl['p_open'] = (25 / (df_bowl['economy'] + 1) + df_bowl['Wickets'] * 10)
    df_bowl['p_mid'] = (df_bowl['Dots'] * 1.5 + df_bowl['Wickets'] * 10)
    df_bowl['p_death'] = (df_bowl['Wickets'] * 25)
    # Consolidate Match-level data
    bat_sub = pd.DataFrame({'pid': df_bat['playerid'], 'mid': df_bat['match_id'],
                           'p_open': df_bat['p_open'], 'p_mid': df_bat['p_mid'], 'p_death': df_bat['p_death']})
    bowl_sub = pd.DataFrame({'pid': df_bowl['Bowler_id'], 'mid': df_bowl['matchId'],
                             'p_open': df_bowl['p_open'], 'p_mid': df_bowl['p_mid'], 'p_death': df_bowl['p_death']})
   
    all_match = pd.concat([bat_sub, bowl_sub], ignore_index=True).sort_values(['pid', 'mid'])
    # --- 3. COMPUTE RECENCY & VOLATILITY PER PHASE ---
    phases = ['p_open', 'p_mid', 'p_death']
    for p in phases:
        all_match[f'{p}_ema'] = all_match.groupby('pid')[p].transform(lambda x: x.ewm(span=5).mean())
        all_match[f'{p}_std'] = all_match.groupby('pid')[p].transform(lambda x: x.rolling(5, min_periods=1).std())
    # --- 4. MERGE TO FULL ROSTER ---
    stats = all_match.groupby('pid').last().reset_index()
    master = master.merge(stats, left_on='Player_id', right_on='pid', how='left').fillna(0)
    # --- 5. SCALE SCORES & CALCULATE FINAL ---
    SELECTION_BASE = 30
    mapping = {'p_open': 'Opening_Score', 'p_mid': 'Middle_Score', 'p_death': 'Death_Score'}
   
    for raw, name in mapping.items():
        raw_val = master[f'{raw}_ema'] / master['level_res']
        active_mask = raw_val > 0
        if active_mask.any():
            mu, sig = raw_val[active_mask].mean(), raw_val[active_mask].std()
            master[name] = SELECTION_BASE + (70 / (1 + np.exp(-((raw_val - mu) / (sig + 1e-9)))))
            # Consistency Weighting (EMA / Volatility)
            master[f'{name}_con'] = master[f'{raw}_ema'] / (master[f'{raw}_std'] + 1)
        else:
            master[name] = SELECTION_BASE
            master[f'{name}_con'] = 0
    # Weighted Average Final Score based on Consistency
    con_sum = master['Opening_Score_con'] + master['Middle_Score_con'] + master['Death_Score_con'] + 1e-9
    master['Final_Phase_Score'] = (
        (master['Opening_Score'] * master['Opening_Score_con']) +
        (master['Middle_Score'] * master['Middle_Score_con']) +
        (master['Death_Score'] * master['Death_Score_con'])
    ) / con_sum
    # --- 6. ADDITIVE VERSATILITY BONUS (+10 Points) ---
    # Case-insensitive role check
    role_col = 'Role' if 'Role' in master.columns else 'role'
   
    # Bonus 1: Openers/Top order performing at the Death
    master.loc[(master['Opening_Score'] > 90) & (master['Death_Score'] > 90), 'Final_Phase_Score'] += 10
   
    # Bonus 2: Bowlers (usually Death specialists) performing in the Opening phase
    if role_col in master.columns:
        is_bowler = master[role_col].str.lower().str.contains('bowl', na=False)
        master.loc[is_bowler & (master['Opening_Score'] > 90) & (master['Death_Score'] > 90), 'Final_Phase_Score'] += 10
   
    # Cap Final Score at 100
    master['Final_Phase_Score'] = master['Final_Phase_Score'].clip(upper=100)
    # Ground inactive players at 30
    master.loc[master['p_open_ema'] == 0, ['Opening_Score', 'Middle_Score', 'Death_Score', 'Final_Phase_Score']] = SELECTION_BASE
   
    # Return master with original columns plus the 4 new phase columns
    new_cols = ['Opening_Score', 'Middle_Score', 'Death_Score', 'Final_Phase_Score']
    return master[list(df_player.columns) + new_cols]

def calculate_role_specific_impact(df_bat, df_bowl, df_player):
    # --- 1. CLEAN COLUMN NAMES ---
    # Strip spaces and normalize to avoid KeyErrors
    df_player.columns = [c.strip() for c in df_player.columns]
    master = df_player.copy()
   
    # 2. Level Resistance Mapping
    level_map = {'Uncapped': 0.5, 'Capped': 1.0, 'Elite': 1.5}
    master['level_res'] = master['Level'].map(level_map).fillna(1.0)
   
    # 3. Match-Level Impact Calculation
    # Batting Impact: Runs weighted by SR + boundary bonuses
    df_bat['bat_imp'] = (df_bat['Runs'] * (df_bat['strike Rate'] / 100)) + (df_bat['sixes'] * 2) + df_bat['fours']
    bat_agg = df_bat.groupby('playerid')['bat_imp'].mean().reset_index()
   
    # Bowling Impact: Wicket value + Dot ball pressure - Economy penalty
    df_bowl['bowl_imp'] = (df_bowl['Wickets'] * 25) + df_bowl['Dots'] - (df_bowl['economy'] * 5)
    bowl_agg = df_bowl.groupby('Bowler_id')['bowl_imp'].mean().reset_index()
   
    # 4. Merge to Master
    master = master.merge(bat_agg, left_on='Player_id', right_on='playerid', how='left').fillna(0)
    master = master.merge(bowl_agg, left_on='Player_id', right_on='Bowler_id', how='left').fillna(0)
   
    # --- 5. ROBUST ROLE-CHECK LOGIC ---
    # Identify which role column exists (Role or role)
    available_cols = [c.lower() for c in master.columns]
    if 'role' in available_cols:
        # Find the actual case-sensitive name of the role column
        actual_role_col = [c for c in master.columns if c.lower() == 'role'][0]
    else:
        # Fallback if the column is missing: default to 'Batter' to prevent crash
        master['Role_Temp'] = 'Batter'
        actual_role_col = 'Role_Temp'
    def compute_raw_rsi(row):
        role_val = str(row[actual_role_col]).lower()
        if 'all' in role_val:  # All-rounder
            raw = (row['bat_imp'] * 0.6) + (row['bowl_imp'] * 0.6)
        elif 'bowl' in role_val:  # Bowler
            raw = row['bowl_imp']
        else:  # Batter / WK
            raw = row['bat_imp']
        return raw / row['level_res']
    master['raw_rsi'] = master.apply(compute_raw_rsi, axis=1)
   
    # --- 6. SIGMOID SCALING (30-100 BASELINE) ---
    SELECTION_BASE = 30
    active_mask = master['raw_rsi'] > 0
   
    if active_mask.any():
        mu = master.loc[active_mask, 'raw_rsi'].mean()
        sigma = master.loc[active_mask, 'raw_rsi'].std()
        master['z'] = (master['raw_rsi'] - mu) / (sigma + 1e-9)
        master['RSI_Score'] = SELECTION_BASE + (70 / (1 + np.exp(-master['z'])))
    else:
        master['RSI_Score'] = SELECTION_BASE
       
    master.loc[~active_mask, 'RSI_Score'] = SELECTION_BASE
   
    # Clean up temporary columns
    if 'Role_Temp' in master.columns:
        master = master.drop(columns=['Role_Temp'])
       
    return master[list(df_player.columns) + ['RSI_Score']]

def calculate_tcs_final_optimized(df_bat, df_bowl, df_league, df_player):
    # --- 1. PRE-PROCESSING ---
    # Standardizing columns based on your provided Data Model
    for df in [df_bat, df_bowl, df_league, df_player]:
        df.columns = [c.strip() for c in df.columns]
   
    master = df_player.copy()
    level_map = {'Uncapped': 0.5, 'Capped': 1.0, 'Elite': 1.5}
    master['level_res'] = master['Level'].map(level_map).fillna(1.0)
    # --- 2. BATTING CONTRIBUTION ---
    # Team total runs per match
    df_bat['team_total_runs'] = df_bat.groupby(['match_id', 'Team_Name'])['Runs'].transform('sum')
    df_bat['rel_runs'] = df_bat['Runs'] / (df_bat['team_total_runs'] + 1e-9)
   
    # Entry Point Stress (EPW):
    # Logic: If a player arrives early (Low 'Over of wicket' of previous batters)
    # and many wickets have fallen ('Fall of Wicket'), the stress is higher.
    # We use 'Fall of Wicket' and 'Over of wicket' from your batting schema.
    df_bat['epw'] = (df_bat['Fall of Wicket'].fillna(0) + 1) / (df_bat['Over of wicket'].fillna(0) + 1)
   
    # Merge with League for results and importance
    bat_m = df_bat.merge(df_league[['match_id', 'home team', 'visiting team',
                                   'home_importance', 'visitor_importance', 'results']],
                             on='match_id', how='left')
   
    # Result Weighting: 1.2 for Win, 0.8 for Loss, 1.0 for No Result/Tie
    def get_res_factor(team, res_str):
        res = str(res_str).lower()
        if 'no result' in res or 'tie' in res: return 1.0
        return 1.2 if team.lower() in res else 0.8
    bat_m['win_factor'] = bat_m.apply(lambda x: get_res_factor(x['Team_Name'], x['results']), axis=1)
    bat_m['m_imp'] = np.where(bat_m['Team_Name'] == bat_m['home team'],
                              bat_m['home_importance'], bat_m['visitor_importance'])
   
    # Exponential Stakes Calculation
    bat_m['raw_tcs'] = (bat_m['rel_runs'] * bat_m['win_factor'] * bat_m['epw'] * np.exp(bat_m['m_imp']/5))
   
    bat_gen = bat_m.groupby('playerid')['raw_tcs'].mean()
    bat_big = bat_m[bat_m['m_imp'] > 7].groupby('playerid')['raw_tcs'].mean()
    # --- 3. BOWLING CONTRIBUTION ---
    # Using 'matchId' and 'Team' from your bowling schema
    df_bowl['team_total_wkts'] = df_bowl.groupby(['matchId', 'Team'])['Wickets'].transform('sum')
    df_bowl['rel_wkts'] = df_bowl['Wickets'] / (df_bowl['team_total_wkts'] + 1e-9)
   
    # Innings Context (Innings 2 is high pressure/defending)
    df_bowl['epw'] = df_bowl['Innings'].map({1: 1.0, 2: 1.2}).fillna(1.0)
   
    bowl_m = df_bowl.merge(df_league[['match_id', 'home team', 'visiting team',
                                     'home_importance', 'visitor_importance', 'results']],
                          left_on='matchId', right_on='match_id', how='left')
   
    bowl_m['win_factor'] = bowl_m.apply(lambda x: get_res_factor(x['Team'], x['results']), axis=1)
    bowl_m['m_imp'] = np.where(bowl_m['Team'] == bowl_m['home team'],
                               bowl_m['home_importance'], bowl_m['visitor_importance'])
   
    bowl_m['raw_tcs'] = (bowl_m['rel_wkts'] * bowl_m['win_factor'] * bowl_m['epw'] * np.exp(bowl_m['m_imp']/5))
   
    bowl_gen = bowl_m.groupby('Bowler_id')['raw_tcs'].mean()
    bowl_big = bowl_m[bowl_m['m_imp'] > 7].groupby('Bowler_id')['raw_tcs'].mean()
    # --- 4. INTEGRATION ---
    master = master.merge(bat_gen.rename('bat_tcs'), left_on='Player_id', right_index=True, how='left').fillna(0)
    master = master.merge(bowl_gen.rename('bowl_tcs'), left_on='Player_id', right_index=True, how='left').fillna(0)
    master = master.merge(bat_big.rename('bat_big'), left_on='Player_id', right_index=True, how='left').fillna(0)
    master = master.merge(bowl_big.rename('bowl_big'), left_on='Player_id', right_index=True, how='left').fillna(0)
    # Combine Raw Scores and apply Level Resistance
    master['final_raw'] = (master['bat_tcs'] + master['bowl_tcs']) / master['level_res']
    master['big_match_avg'] = (master['bat_big'] + master['bowl_big']) / master['level_res']
    # --- 5. SCALING & BIG-MATCH PENALTY ---
    SELECTION_BASE = 30
    active = master['final_raw'] > 0
    if active.any():
        mu, sigma = master.loc[active, 'final_raw'].mean(), master.loc[active, 'final_raw'].std()
        master['z'] = (master['final_raw'] - mu) / (sigma + 1e-9)
        master['TCS_Score'] = SELECTION_BASE + (70 / (1 + np.exp(-master['z'])))
    else:
        master['TCS_Score'] = SELECTION_BASE
    # Big-Match Penalty: -15 if player chokes (Avg in Imp > 7 matches < 70% of Season Avg)
    penalty_mask = (master['big_match_avg'] > 0) & (master['big_match_avg'] < (master['final_raw'] * 0.7))
    master.loc[penalty_mask, 'TCS_Score'] -= 15
   
    master['TCS_Score'] = master['TCS_Score'].clip(lower=SELECTION_BASE, upper=100)
    master.loc[~active, 'TCS_Score'] = SELECTION_BASE
    return master[list(df_player.columns) + ['TCS_Score']]

def calculate_fielding_impact_clutch(df_field, df_league, df_player, df_bat):
    # --- 1. CLEANING & COLUMN NORMALIZATION ---
    for df in [df_field, df_league, df_player, df_bat]:
        df.columns = [c.strip() for c in df.columns]
   
    master = df_player.copy()
    level_map = {'Uncapped': 0.5, 'Capped': 1.0, 'Elite': 1.5}
    master['level_res'] = master['Level'].map(level_map).fillna(1.0)
    # Function to find the match ID column regardless of capitalization
    def get_mid_col(df):
        cols = [c for c in df.columns if c.lower() == 'match_id' or c.lower() == 'matchid']
        return cols[0] if cols else None
    mid_field = get_mid_col(df_field)
    mid_league = get_mid_col(df_league)
    mid_bat = get_mid_col(df_bat)
    # --- 2. WICKET IMPORTANCE (The "Big Catch" Value) ---
    # Using the dismissed batsman's match-level impact as the 'Importance of Wicket'
    df_bat['bat_val'] = (df_bat['Runs'] * (df_bat['strike Rate'] / 100))
    # Normalize to a scale of 0.5 - 2.5
    mu_bat = df_bat['bat_val'].mean()
    df_bat['wicket_val'] = 0.5 + (df_bat['bat_val'] / (mu_bat + 1e-9))
   
    target_batsman_value = df_bat.groupby('Player Name')['wicket_val'].mean().reset_index()
    # --- 3. MERGE FIELDING WITH CONTEXT ---
    # Merge with dismissed batsman value
    field_m = df_field.merge(target_batsman_value, left_on='batsman', right_on='Player Name', how='left').fillna(1.0)
   
    # Identify importance columns
    h_imp = 'home_importance' if 'home_importance' in df_league.columns else 'home importance'
    v_imp = 'visitor_importance' if 'visitor_importance' in df_league.columns else 'visitor importance'
   
    # Merge with League Data using standard match_id
    field_m = field_m.merge(df_league[[mid_league, 'home team', 'visiting team', h_imp, v_imp]],
                            left_on=mid_field, right_on=mid_league, how='left')
    # Calculate Match Stakes for the fielder's specific team
    field_m['m_imp'] = np.where(field_m['Feilder Team'] == field_m['home team'], field_m[h_imp], field_m[v_imp])
   
    # --- 4. RAW IMPACT CALCULATION ---
    # Catch Difficulty: Hard (3.5), Medium (1.8), Easy (1.0)
    catch_weight = {'Hard': 3.5, 'Medium': 1.8, 'Easy': 1.0}
    field_m['cd_score'] = field_m['Catch Level'].map(catch_weight).fillna(1.0)
   
    # Formula: Catch Difficulty * Wicket Importance * exp(Match Importance / 5)
    field_m['raw_f_impact'] = field_m['cd_score'] * field_m['wicket_val'] * np.exp(field_m['m_imp'] / 5)
   
    field_agg = field_m.groupby('fielder_id')['raw_f_impact'].sum().reset_index()
    # --- 5. ROSTER INTEGRATION & SCALING ---
    master = master.merge(field_agg, left_on='Player_id', right_on='fielder_id', how='left').fillna(0)
    master['adj_f_impact'] = master['raw_f_impact'] / master['level_res']
    SELECTION_BASE = 30
    active = master['adj_f_impact'] > 0
    if active.any():
        mu_f, sigma_f = master.loc[active, 'adj_f_impact'].mean(), master.loc[active, 'adj_f_impact'].std()
        master['z'] = (master['adj_f_impact'] - mu_f) / (sigma_f + 1e-9)
        master['FIS_Score'] = SELECTION_BASE + (70 / (1 + np.exp(-master['z'])))
    else:
        master['FIS_Score'] = SELECTION_BASE
    master.loc[~active, 'FIS_Score'] = SELECTION_BASE
    master['FIS_Score'] = master['FIS_Score'].clip(lower=SELECTION_BASE, upper=100)
    # Return only the player master columns + the new FIS_Score
    return master[list(df_player.columns) + ['FIS_Score']]

def calculate_master_composite_score(df_bat, df_bowl, df_league, df_field, df_player):
    """
    Master module to compute the final composite score by integrating all 6 sub-modules.
    Updated for exact data model compatibility:
    - Column names match: e.g., 'match_id', 'playerid', 'Bowler_id', 'fielder_id', 'Player_id', etc.
    - Relationships enforced via merges on exact keys.
    - All DataFrames have columns stripped of leading/trailing spaces.
    - Weights: Equal (1/6) for balance; adjust if specific formula provided.
    - Final score clipped to 30-100.
    - Returns full player details + all sub-scores + Overall_Score, sorted descending.
    """
    # Standardize all input DataFrames (strip column names)
    for df in [df_bat, df_bowl, df_league, df_field, df_player]:
        df.columns = [c.strip() for c in df.columns]

    # Step 1: Compute sub-scores (each returns a DF with Player_id and score)
    # PPS (Pressure Performance Score)
    pps_df = calculate_professional_pps(df_bat, df_bowl, df_league, df_player.copy())
    pps_df = pps_df[['Player_id', 'PPS_Score']]

    # Consistency Score
    cons_df = calculate_consistency_with_form(df_bat, df_bowl, df_player.copy())
    cons_df = cons_df[['Player_id', 'Consistency_Score']]

    # Phase Scores (using Final_Phase_Score)
    phase_df = calculate_phased_final(df_bat, df_bowl, df_player.copy())
    phase_df = phase_df[['Player_id', 'Final_Phase_Score']]

    # RSI (Role Specific Impact)
    rsi_df = calculate_role_specific_impact(df_bat, df_bowl, df_player.copy())
    rsi_df = rsi_df[['Player_id', 'RSI_Score']]

    # TCS (Team Contribution Score)
    tcs_df = calculate_tcs_final_optimized(df_bat, df_bowl, df_league, df_player.copy())
    tcs_df = tcs_df[['Player_id', 'TCS_Score']]

    # FIS (Fielding Impact Score)
    fis_df = calculate_fielding_impact_clutch(df_field, df_league, df_player.copy(), df_bat)
    fis_df = fis_df[['Player_id', 'FIS_Score']]

    # Step 2: Merge all to master player DF
    master = df_player.copy()
    for score_df in [pps_df, cons_df, phase_df, rsi_df, tcs_df, fis_df]:
        master = master.merge(score_df, on='Player_id', how='left')

    # Fill NaN with baseline 30
    score_cols = ['PPS_Score', 'Consistency_Score', 'Final_Phase_Score', 'RSI_Score', 'TCS_Score', 'FIS_Score']
    master[score_cols] = master[score_cols].fillna(30)

    # Step 3: Weighted average for Overall_Score (equal weights)
    weights = {col: 1/6 for col in score_cols}
    master['Overall_Score'] = sum(master[col] * weights[col] for col in score_cols)
    master['Overall_Score'] = master['Overall_Score'].clip(lower=30, upper=100)

    # Step 4: Return sorted master
    return_cols = list(df_player.columns) + score_cols + ['Overall_Score']
    return master[return_cols].sort_values(by='Overall_Score', ascending=False)

# Example Execution (load data and run - adjust paths if needed)
# Assuming CSVs are in current directory
# league = pd.read_csv('league.csv')
# new_batting = pd.read_csv('new_batting.csv')
# new_bowling = pd.read_csv('new_bowling.csv')
# new_fielding = pd.read_csv('new_fielding.csv')
# player_details = pd.read_csv('player_details.csv')
# master_leaderboard = calculate_master_composite_score(new_batting, new_bowling, league, new_fielding, player_details)
print(f"Total players: {len(master_leaderboard)}")
print(master_leaderboard.head().to_string(index=False))